**Number Theory Assignment**

---
IMT2021023 - Munagala Kalyan Ram

IMT2021516 - Puneet Kalyan Upadrasta


In [2]:
!pip install gmpy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.7 MB/s eta 0:00:00


In [48]:
#Main code
import gmpy2
from gmpy2 import mpz,mpfr, f_mod, add,mul,is_even,sub, div, powmod, is_prime
from random import seed
from random import randint
from math import floor
gmpy2.set_context(gmpy2.context())

#Functions to be used in Main Code
def EGCD(a,b): #binary EGCD using gmpy library and V.shoup as reference

  r1,r2,e=mpz(a),mpz(b),mpz(0)
  while(is_even(r1) and is_even(r2)):
    r1,r2,e=r1>>1,r2>>1, add(e,1)

  ap,bp,s,t,sp,tp=r1,r2,1,0,0,1 #sp,tp are s' and t'
  
  while(r2>0):
    while(is_even(r1)):
      r1=r1>>1
      if(is_even(s) and is_even(t)):
          s,t=s>>1,t>>1
      else:
        s,t=add(s, bp)>>1,sub(t, ap)>>1
    
    while(is_even(r2)):
      r2=r2>>1
      if(is_even(sp) and is_even(tp)):
        sp,tp=sp>>1,tp>>1
      else:
        sp,tp=add(sp, bp)>>1,sub(tp,ap)>>1
    
    if(r1>r2):
      r1,s,t,r2,sp,tp=r2,sp,tp,r1,s,t

    r2,sp,tp=sub(r2,r1), sub(sp,s), sub(tp,t)
 
  return (mul(pow(2,e),r1),s,t)


def CRT(alist,nlist,k): #Reference from V.shoup, passing ai's and ni's as paramenters
  n,a=mpz(1),mpz(0)

  for i in range(k): 
    n=mul(n,nlist[i])

  elist=[]
  
  for i in range(k):
    np=1
    for j in range(k):
      if(j!=i):
        np=mul(np,nlist[j]) # np is n/ni
    b=powmod(np,1,nlist[i])
    inverse=EGCD(nlist[i],np) #Running EGCD to get inverse
    t=powmod(inverse[2], 1, nlist[i])
    elist.append(mul(np,t))

  for i in range(k):
    a=add(a,mul(alist[i],elist[i])) #multiplying ai*ei and adding them up
  
  return powmod(a,1,n)

def GeneratePrime(llimit, ulimit,k, option): #generates different list of numbers based on option given
  seed(1)
  lprimes,i=[],0
  if(option==1):
    for p in (range(llimit,ulimit)): #generates a list of k unique primes in a given range
      if(is_prime(p) and p not in lprimes):
        if(i<k):
          lprimes.append(p)
          i+=1
        else:
          break

  elif(option==0): #generates k unique numbers in a given range
    while(i<k):
      x=mpz(randint(llimit,ulimit))
      if(x not in lprimes):
        lprimes.append(x)
        i+=1
  
  elif(option==2): #generates any k numbers in a given range
    for i in range(k):
      lprimes.append(mpz(randint(llimit,ulimit)))
  return lprimes

def egcd(a, b): #normal egcd which is used for finding EEA({ri,si,ti}) for all i in 0 to l+1
  EEA=[]
  r,rp=mpz(a),mpz(b)
  s,sp,t,tp=mpz(1),mpz(0),mpz(0),mpz(1)
  while(rp>0):
    q=mpz(floor(r/rp))
    rpp=r%rp
    EEA.append((r,s,t))
    r,s,t,rp,sp,tp=rp,sp,tp,rpp,sub(s,mul(sp,q)),sub(t,mul(tp,q))

  d=r
  EEA.append((r,s,t))
  return EEA

#Globals Known to Transmitter and Receiver

GM=mpz(2)
Gu=mpfr(0.5)
Gprimes=[]
Gk=0
Gp=mpz(1)
Gchoice=[]

#Functions implemented from Main Code
def GlobalSetup():

  global Gu,GM #taking M and u as input
  print("Enter x such that M=2^x:", end=" ")
  x=mpz(input())
  GM=2**x
  
  print("Enter u:", end=" ")
  Gu=mpfr(input()) #Value should be b/w 0<u<1

  global Gk,Gprimes,Gp

  alength=GM.bit_length()
  primes=GeneratePrime(2,(2**16)-1,510,1)
  product=mpz(1)

  Gk,Gp,sizeGp=0,mpz(1),0
  Gprimes=[]

  while(Gk<500): #finding the value of k by checking at which value of k n> 2MP^2 and primes are taken from 2,3 ...

    Gk+=1
    llimit=mpz(float(Gu)*int(Gk))
    Gprimes.append(primes[Gk-1])
    product=mul(product,primes[Gk-1])
    Gp=1
    for i in range(1,llimit+1):
      Gp=mul(Gp,Gprimes[-i])

    if(product>2*GM*(Gp**2)):
      break

  global Gchoice
  Gchoice=GeneratePrime(0,int(float(Gu)*int(Gk)), 50, 2)

def Transmit(alist,count): 
  seed(20)
  l=Gchoice[count] # selecting a number from a list of random numbers
  print("l selected: %d" % (l))
  Indexes,blist=GeneratePrime(0,Gk,l,0),[]
 
  for i in range(0,Gk): #corrupts the ai's at l random indexes
    if(i in Indexes):
      while(1):
        x=mpz(randint(0,Gprimes[i]-1))
        if(x!=alist[i]):
          blist.append(x)
          break
      
    else:
      blist.append(alist[i])

  return blist

def ReedSolomonSend(a, count):
  alist=[]
  for i in range(Gk):
    alist.append(powmod(a,1,Gprimes[i]))

  return Transmit(alist,count)

def ReedSolomonReceive(blist): #gets the corrupted bi's list
  b=CRT(blist,Gprimes, Gk) #running CRT to get b from bi's and ni's
 
  n=mpz(1)
  for i in Gprimes:
    n=mul(n,i)
  
  EEAlist=egcd(n,b) 
  rnew=mul(GM,Gp) #using rational reconstruction concept to find the r' and t' to get a0
  tnew=Gp
  rfinal,tfinal=0,0

  for gcd in EEAlist:
    if(gcd[0]<=rnew ):
      rfinal, tfinal=gcd[0] , gcd[2]
      break
  
  if(rfinal>0 and tfinal>0 and rfinal%tfinal==0 ):
    return rfinal/tfinal
  else:
    return -1

if __name__=="__main__": 
  
  #Integer version of Reed-Solomon Error Correcting Code
  GlobalSetup()
  count=0
  while(1):
     print("Enter the Message to be transmitted:", end=" ")
     s=mpz(input())
     if(s==-1 or s>GM): #Enter -1 to exit or program will exit if message bound is greater than 'GM'
      break
     blist=ReedSolomonSend(s,count) 
     count+=1
     result=ReedSolomonReceive(blist)
     if(result>0):
      print("Successfully recovered message: %d" % (result))
     else:
      print("Failed to recover message" )

Enter x such that M=2^x: 230
Enter u: 0.1
Enter the Message to be transmitted: 3253246230581012034032
l selected: 1
Successfully recovered message: 3253246230581011808256
Enter the Message to be transmitted: 8772378528391923452443
l selected: 4
Successfully recovered message: 8772378528391923499008
Enter the Message to be transmitted: 890932475092374067104726191702016
l selected: 0
Successfully recovered message: 890932475092374067104726191702016
Enter the Message to be transmitted: 567697698123684907719283751873467012
l selected: 2
Successfully recovered message: 567697698123684934275455912010317824
Enter the Message to be transmitted: 879012734098709
l selected: 0
Successfully recovered message: 879012734098709
Enter the Message to be transmitted: 23458929084375
l selected: 3
Successfully recovered message: 23458929084375
Enter the Message to be transmitted: 8932170498713094875327098572983645876230487956792601489
l selected: 3
Failed to recover message
Enter the Message to be transmi

**Test Case 1:**

x=123

u=0.4

*   2344253212
*   23465475123312335
*   78547962514323451
*   9009234858123412232135126123
*   34265361346235743123
*   -1 

Result:

* Success
* Fail
* Success
* Success
* Success


**Test Case 2:**

x=230

u=0.1

* 3253246230581012034032
* 8772378528391923452443
* 890932475092374067104726191702016
* 567697698123684907719283751873467012
* 879012734098709
* 23458929084375
* 8932170498713094875327098572983645876230487956792601489
* -1

Result:

* Success
* Success
* Success
* Success
* Success
* Success
* Fail





